In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import re
import requests

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import datefinder

from mlib.ds.dataframes import df_count

from parse_players import parse_players

In [3]:
PLAYER = "Cannonball Adderley"

In [4]:
html = requests.get("https://www.jazzdisco.org/cannonball-adderley/discography/")
soup = BeautifulSoup(html.text, 'html.parser')

In [5]:
a = soup.prettify().split('<!-- id="start-here" -->')[1]
b = a.split('<!-- id="ends-here" -->')[0]

html_segments = b.split("<h3>")

In [11]:
print(html_segments[56])


       <a href="./session-index/#590202" name="590202">
        Paul Chambers Quintet
       </a>
      </h3>
      <p>
       Freddie Hubbard, trumpet; Cannonball Adderley, alto sax; Wynton Kelly, piano; Paul Chambers, bass; Jimmy Cobb, drums.
      </p>
      <p class="date">
       Universal Recording, Chicago, IL, February 2, 1959
      </p>
      <table width="100%">
       <tr>
        <td width="15%">
         tk.2
         <td width="35%">
          Ease It (alternate take)
          <td>
           Mosaic MD6-205
           <tr>
            <td>
             59-1073 tk.5
             <td>
              Ease It
              <td>
               Vee-Jay VJLP 1014
               <tr>
                <td>
                 tk.6
                 <td>
                  Just Friends (alternate take)
                  <td>
                   Vee-Jay (J) 22YB-2088; Mosaic MD6-205
                   <tr>
                    <td>
                     59-1074 tk.8
                     <td

In [12]:
s = BeautifulSoup(html_segments[56], 'html.parser')

session_name = s.a.get_text().strip()
session_number = s.a["name"]
address_date = s.find('p', attrs={'class': 'date'}).get_text().strip()

# date = list(datefinder.find_dates(address_date))[0].strftime("%Y-%m-%d")
# state = re.findall(r"\b([A-Z]{2})[,\b]", address_date)[0]

session = pd.DataFrame({
    "player": PLAYER,
    "session_id": session_number,
    "session": session_name,
    "address_date": address_date
}, index=[0])

session

,player,session_id,session,address_date
0,Cannonball Adderley,590202,Paul Chambers Quintet,"Universal Recording, Chicago, IL, February 2, ..."


In [16]:
recordings = pd.read_html(s.table.prettify())[0]
recordings.columns = ["recording_id", "name", "release_id"]

recordings["recording_id"] = np.arange(1, recordings.shape[0] + 1)

recordings = recordings.replace("-", np.nan)

recordings["release_id"] = recordings.release_id.str.split(r"[,;]\s")
recordings = recordings.explode("release_id")

recordings.insert(0, "session_id", session_number)

recordings

,session_id,recording_id,name,release_id
0,590202,1,Ease It (alternate take),Mosaic MD6-205
1,590202,2,Ease It,Vee-Jay VJLP 1014
2,590202,3,Just Friends (alternate take),Vee-Jay (J) 22YB-2088
2,590202,3,Just Friends (alternate take),Mosaic MD6-205
3,590202,4,Just Friends,Vee-Jay VJLP 1014
4,590202,5,I Got Rhythm,Vee-Jay (J) 22YB-2088
4,590202,5,I Got Rhythm,Mosaic MD6-205
5,590202,6,NaN,Vee-Jay VJLP 1014


In [10]:
players = s.p.get_text().strip()

players = parse_players(players)

all_recordings = ", ".join([str(x) for x in list(range(1, recordings.recording_number.max() + 1))])
players["recording_number"] = players.recording_number.fillna(all_recordings)
players["recording_number"] = players.recording_number.str.split(", ")
players = players.explode("recording_number")
players["recording_number"] = players.recording_number.astype(int)

players["instrument"] = players.instrument.str.split(", ")
players = players.explode("instrument")

players

,player_name,instrument,recording_number
0,Nat Adderley,cornet,1
0,Nat Adderley,cornet,2
0,Nat Adderley,cornet,3
0,Nat Adderley,cornet,4
0,Nat Adderley,cornet,5
1,Cannonball Adderley,alto sax,1
1,Cannonball Adderley,alto sax,2
1,Cannonball Adderley,alto sax,3
1,Cannonball Adderley,alto sax,4
1,Cannonball Adderley,alto sax,5


In [72]:
releases = [x.get_text() for x in s.find_all("p")][-1]
releases = [x.split("\n", maxsplit=1) for x in releases.split("*") if x.strip() != ""]
releases = [(i.strip(), j.strip()) for i,j in releases]
releases = [(i, j) if " - " in j else (i, j + " - ") for i,j in releases]
releases = [(i, *tuple(j.split(" - ", maxsplit=1))) for i,j in releases]

releases = pd.DataFrame(releases, columns=["release_id", "release_artist", "release_name"])
releases = releases.replace("", np.nan)

releases.insert(0, "session_id", session_number)



releases

,session_id,release_id,release_artist,release_name
0,560718,EmArcy MG-36100,To The Ivy League From Nat Adderley\n \n...,NaN


In [12]:
session_export_name = f"exports/SESSION_{session_number}.csv"
recording_export_name = f"exports/RECORDING_{session_number}.csv"
player_export_name = f"exports/PLAYER_{session_number}.csv"
release_export_name = f"exports/RELEASE_{session_number}.csv"

session.to_csv(session_export_name, index=False)
recordings.to_csv(recording_export_name, index=False)
players.to_csv(player_export_name, index=False)
releases.to_csv(release_export_name, index=False)

In [54]:
re.sub(r"[^\d,-]", "", "1,4-3,3.asdkn1,4")

'1,4-3,31,4'